In [22]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import datetime
from tqdm import tqdm

import torch
from torch import nn
import torch.nn.functional as func
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim

from models import VGG
from train import train1Epoch
from train import test1Epoch
from datasetCreator import ImageSubset
torch.cuda.empty_cache()
import seaborn as sns

import os
import cv2

In [23]:
!rm -v -f runs\*
!rm -r runs

rm: cannot remove 'runs': No such file or directory


In [24]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
torch.backends.cudnn.benchmark = True
#writer=SummaryWriter()

In [26]:
ssh -N -f -L localhost:16006:localhost:6006 ddavilag@dsmlp-login.ucsd.edu

SyntaxError: invalid syntax (1158549507.py, line 1)

In [29]:
tensorboard --logdir <path> --port 16006

/bin/bash: path: No such file or directory


In [5]:
#https://stackoverflow.com/questions/43837372/how-does-one-open-a-tensorboard-port-in-linux
%tensorboard --logdir <path-to-your-logs> --path_prefix /tensorboard

In [30]:
#!start chrome http://localhost:6006/
!xdg-open http://localhost:6006

/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'http://localhost:6006'


In [7]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [8]:
#pip install https://download.pytorch.org/whl/nightly/cu116/torch-1.14.0.dev20221023%2Bcu116-cp39-cp39-win_amd64.whl
#pip install https://download.pytorch.org/whl/nightly/cu116/torchvision-0.15.0.dev20221023%2Bcu116-cp39-cp39-win_amd64.whl
torch.cuda.is_available(), torch.version.cuda

(True, '11.1')

In [9]:
DATA_PATH = '/home/ddavilag/teams/dsc-180a---a14-[88137]/df_bnpp_datapaths.csv'
KEY_PATH = '/home/ddavilag/teams/dsc-180a---a14-[88137]/df_bnpp_keys.csv'

In [10]:
df_datapaths = pd.read_csv(DATA_PATH, header=None).T.merge(pd.read_csv(KEY_PATH, header=None).T, left_index=True, right_index=True)
df_datapaths.columns = ['filepaths', 'key']
df_datapaths.key = df_datapaths.key.apply(lambda x: eval(x))
df_datapaths.filepaths = df_datapaths.filepaths.apply(lambda x: eval(x))
df_datapaths = df_datapaths.set_index('key')
#missing h5py files 7-9

In [11]:
cols= ['unique_key', 'bnpp_value_log', 'BNP_value' 
        #'BNPP_weight', 'PNA_mask', 'PNA_wight_mask', 'BNP_value', 'age_at_sampletime'
       ]
test_df = pd.read_csv('/home/ddavilag/teams/dsc-180a---a14-[88137]/BNPP_DT_test_with_ages.csv', usecols = cols).set_index('unique_key')
train_df = pd.read_csv('/home/ddavilag/teams/dsc-180a---a14-[88137]/BNPP_DT_train_with_ages.csv', usecols = cols).set_index('unique_key')
val_df = pd.read_csv('/home/ddavilag/teams/dsc-180a---a14-[88137]/BNPP_DT_val_with_ages.csv', usecols = cols).set_index('unique_key')
print(train_df.shape, test_df.shape, val_df.shape)
train_df['heart'] = train_df['BNP_value'].apply(lambda x: int(x > 400))
test_df['heart'] = test_df['BNP_value'].apply(lambda x: int(x > 400))
val_df['heart'] = val_df['BNP_value'].apply(lambda x: int(x > 400))


(21374, 2) (2602, 2) (2692, 2)


In [12]:
train_df = train_df.sort_index().merge(df_datapaths, left_index=True, right_index=True)
test_df = test_df.sort_index().merge(df_datapaths, left_index=True, right_index=True)
val_df = val_df.sort_index().merge(df_datapaths, left_index=True, right_index=True)
print(train_df.shape, test_df.shape, val_df.shape)

(15164, 4) (1823, 4) (1913, 4)


In [13]:
train_df['filepaths'] = train_df['filepaths'].str.replace('jmryan','ddavilag')
test_df['filepaths'] = test_df['filepaths'].str.replace('jmryan','ddavilag')
val_df['filepaths'] = val_df['filepaths'].str.replace('jmryan','ddavilag')
new_valid = pd.concat([test_df,val_df],ignore_index=False)
train_df.shape, test_df.shape, val_df.shape,new_valid.shape

((15164, 4), (1823, 4), (1913, 4), (3736, 4))

In [14]:
15164+1823+1913, 1823+1913
#number of rows that we currently have

(18900, 3736)

In [15]:
train_df.reset_index(names='unique_key',inplace=True)
new_valid.reset_index(names='unique_key',inplace=True)

In [16]:
train_df.head()

,unique_key,bnpp_value_log,BNP_value,heart,filepaths
0,Abachug_50267230_img1,2.621176,418.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
1,Abafouck_52403307_img1,2.071882,118.0,0,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
2,Abagash_52691625_img1,1.698101,49.9,0,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
3,Abakleem_50725934_img1,4.301659,20029.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
4,Abaphos_51936331_img1,3.219323,1657.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...


In [17]:
new_valid.head()

,unique_key,bnpp_value_log,BNP_value,heart,filepaths
0,Abadik_50217497_img1,3.334655,2161.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
1,Abdisey_52768446_img1,2.451786,283.0,0,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
2,Abenog_51901728_img1,2.647383,444.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
3,Abeyned_51238065_img1,2.869232,740.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...
4,Abibey_51056225_img1,2.710117,513.0,1,/home/ddavilag/teams/dsc-180a---a14-[88137]/bn...


In [18]:
class PreprocessedImageDataset(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.df = df
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.loc[idx,:]
        filepath = row[4]  #image location
        val = row[1] #bnpp value log
        heart = row[3] #threshold for edema
        
        im = torch.load(filepath)
        #print(filepath, val, heart)
        #plt.imshow(im,cmap='gray')
        #plt.show()
        return im.view(1, 224, 224).expand(3, -1, -1), val#, heart

In [19]:
batch_size = 32 #maybe change to 16, 8
num_workers = 2
pin_memory = True
# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5), std=(0.5))
#     ])

train_set = PreprocessedImageDataset(df=train_df) #, transform = train_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

# valid_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5), std=(0.5))
#     ])

valid_set = PreprocessedImageDataset(df=new_valid) #, transform = valid_transform)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

#print(train_set[0])
#print(iter(train_loader).next())
#print(train.__getitem__(0))
#len(valid_set)

In [20]:
import time
pin_memory = True
print('pin_memory is', pin_memory)
 
for num_workers in range(0, 5, 1): 
    #train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    #sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
    start = time.time()
    for epoch in range(1, 5):
        for i, data in enumerate(train_loader):
            pass
    end = time.time()
    print("Finish with:{} second, num_workers={}".format(end - start, num_workers))

pin_memory is True
Finish with:67.97687101364136 second, num_workers=0


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Finish with:58.32607102394104 second, num_workers=1


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Finish with:32.53528666496277 second, num_workers=2


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 Exception in thread Thread-15:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/lib/python3.9/multiprocessing

RuntimeError: DataLoader worker (pid(s) 4197) exited unexpectedly

In [ ]:
train_set[0], valid_set[0]

In [ ]:
len(train_loader), len(train_set), len(valid_loader),len(valid_set)

In [ ]:
plt.imshow(train_set[0][0].permute(1, 2, 0),cmap='gray')

In [ ]:
plt.imshow(valid_set[0][0].permute(1, 2, 0),cmap='gray')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model = VGG('VGG16').to(device)
model

In [ ]:
# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys_r")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(train_loader)
images, labels = dataiter.next()

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid)
print('  '.join(str('%.2f' % labels[j].item()) for j in range(batch_size)))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO]: Computation device: {device}")

total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"[INFO]: {total_trainable_params:,} trainable parameters.")

# the loss function, Mean Absolute Error
loss_fn = nn.L1Loss().to(device)
#loss_fn = nn.HuberLoss().to(device)
#loss_fn = nn.SmoothL1Loss().to(device)

# the optimizer
# need to tune optimizer
#optimizer = optim.Adam(model.parameters(), lr=0.0001)
LR = 0.0001
optimizer = optim.Adadelta(model.parameters(), lr=LR, rho=0.99)
#optimizer = optim.Adam(model.parameters(), lr=0.0001, amsgrad=True, weight_decay=0.00001)


In [ ]:
torch.cuda.empty_cache()
# Initializing in a separate cell so we can easily add more epochs to the same run
#timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f'runs/VGG16_Ada{LR}_batch{batch_size}_nodes{}')
epoch_number = 0

EPOCHS = 25
#best_vloss = 1_000_000.

# the scheduler
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

tlosses, vlosses=[],[]
writer.add_text('model params', \
                f'''VGG16\n
                loss: {'MAE'}\n
                optimizer: {'Ada'}\n
                \tlearn rate: {LR}\n
                \trho: {RHO}\n
                batch size: {batch_size}\n
                nodes: {500}\n
                dropout: {0.5}\n
                batchnorm: {'yes'}\n
                RELU: {'Leaky'}\n
                epochs: {EPOCHS}
                ''',\
                0)

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    #model.train(True)
    for param in model.parameters():
        param.requires_grad = True
    
    avg_tloss = train1Epoch(epoch_number, model, optimizer, loss_fn, train_loader)
    
    #print(torch.cuda.memory_summary())
    # We don't need gradients on to do reporting
    for param in model.parameters():
        param.requires_grad = False
        
    #model.train(False)
    with torch.no_grad():
        avg_vloss = test1Epoch(epoch_number, model, loss_fn, valid_loader)

    #print(torch.cuda.memory_summary())
    print('LOSS train {} valid {}'.format(avg_tloss, avg_vloss))
    writer.add_scalars('Loss', {'train':avg_tloss,'test':avg_vloss}, epoch)
    tlosses.append(avg_tloss)
    vlosses.append(avg_vloss)

    # Track best performance, and save the model's state
#     if avg_vloss < best_vloss:
#         best_vloss = avg_vloss
#         model_path = 'model_{}_{}'.format(timestamp, epoch_number)
#         torch.save(model.state_dict(), model_path)

    epoch_number += 1
    #scheduler.step(avg_vloss)
writer.flush()
writer.close()

In [ ]:
epochs = np.arange(1,EPOCHS+1)
df = pd.DataFrame(data={'train loss': tlosses,'valid loss':vlosses})
#display(df)
sns.set(style='whitegrid')
#plt.figure(figsize=(20,3))

g = sns.FacetGrid(df, height = 6)

g = g.map(sns.lineplot, x=epochs,y=tlosses,marker='o', label='train')
g = g.map(sns.lineplot, x=epochs,y=vlosses,color='red',marker='o', label='valid')
g.set(ylim=(0, None))
g.add_legend()
plt.xticks(epochs)
plt.show()

In [ ]:
epochs = np.arange(1,EPOCHS+1)
df = pd.DataFrame(data={'train loss': tlosses,'valid loss':vlosses})
#display(df)
sns.set(style='whitegrid')
#plt.figure(figsize=(20,3))

g = sns.FacetGrid(df, height = 6)

g = g.map(sns.lineplot, x=epochs,y=tlosses,marker='o', label='train')
g = g.map(sns.lineplot, x=epochs,y=vlosses,color='red',marker='o', label='valid')
g.set(ylim=(0, None))
g.add_legend()
plt.xticks(epochs)
plt.show()